In [1]:
# Import 
import os
import time
import json
import datetime
import pandas as pd
import numpy as np

from pathlib import Path
from tyssue import Sheet
from tyssue.io import hdf5

from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics import units, effectors, model_factory
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
import matplotlib.pyplot as plt
%matplotlib inline

vispy  won't work


In [3]:
from peripodial.polarity import model
from peripodial.dynamics import EllipsoidLameGeometry as geom

In [4]:
SIM_DIR = Path('/home/lglepin/Simulations/')
today = datetime.date.today()

sim_save_dir = SIM_DIR/f'{today.isoformat()}_YolkElasticity'

try:
    sim_save_dir.mkdir(parents=True, exist_ok=True)
except FileExistsError:
    pass

In [5]:
def init_tissue(hf5_filename='super_eggs_polarity.hdf5',
                json_filename='super_eggs_polarity.json'):
    """
    Initialisation of the superegg tissue
    """
    dsets = hdf5.load_datasets('../examples/' + hf5_filename,
                               data_names=['vert', 'edge', 'face'])

    with open(json_filename, 'r+') as fp:
        specs = json.load(fp)

    sheet = Sheet('spherical', dsets, specs)

    modify_some_initial_settings(sheet)

    return sheet

In [6]:
"""polarity.dynamics to be more clear"""

import numpy as np

from tyssue.geometry.sheet_geometry import ClosedSheetGeometry


class EllipsoidLameGeometry(ClosedSheetGeometry):
    """
    Sphere surrounding the sheet.
    Sphere compress the tissue at its extremity
    """

    @classmethod
    def update_all(cls, eptm):
        cls.center(eptm)
        super().update_all(eptm)

    @staticmethod
    def update_perimeters(eptm):
        """
        Updates the perimeter of each face according to the weight of each junction.
        """
        eptm.edge_df['weighted_length'] = eptm.edge_df.weight * \
            eptm.edge_df.length
        eptm.face_df["perimeter"] = eptm.sum_face(
            eptm.edge_df["weighted_length"])

    @staticmethod
    def normalize_weights(sheet):
        sheet.edge_df["num_sides"] = sheet.upcast_face('num_sides')
        sheet.edge_df["weight"] = sheet.edge_df.groupby('face').apply(
            lambda df: (df["num_sides"] * df["weight"] / df["weight"].sum())
        ).sort_index(level=1).to_numpy()

    @staticmethod
    def update_height(eptm):

        eptm.vert_df["rho"] = np.linalg.norm(
            eptm.vert_df[eptm.coords], axis=1)
        r = eptm.settings['barrier_radius']
        eptm.vert_df["delta_rho"] = (eptm.vert_df["rho"] - r).clip(0)
        eptm.vert_df["height"] = eptm.vert_df["rho"]


In [7]:
"""toolbox.py import in the notebook to be more clear"""

apopto_pattern_kwargs = {'t': 0.,
                         'dt': 1.,
                         'time_of_last_apoptosis': 30.}
# Apoptosis
apoptosis_settings = {
    "critical_area_pulling": 10,
    "critical_area": 0.5,
    "contract_rate": 1.08,
    "basal_contract_rate": 1.01,
    "contract_neighbors": True,
    "contract_span": 3,
    "radial_tension": 50.,
    "max_traction": 30.,
    "current_traction": 0.,
    "geom": geom,
}

# Neighbors propagation
contraction_lt_kwargs = {
    'face_id': -1,
    'face': -1,
    'shrink_rate': 1.05,
    'critical_area': 5.,
    "contraction_column": "line_tension",
    "model": model,
}


# Mechanical perturbation
delaminate_settings = {
    'radial_tension': 80,
    "contract_rate": 1.08,
    "critical_area_pulling": 15,
    "critical_area": 1e-2,
    'current_traction': 0,
    'max_traction': 150,
    'contract_neighbors': True,
    'contract_span': 3,
    'basal_contract_rate': 1.01,
    'geom': geom,
    'model': model}

import json
import random
import os

import numpy as np

from tyssue import Sheet
from tyssue.io import hdf5
from tyssue import config



def modify_some_initial_settings(sheet):
    """
    Need to find an other name function...
    """
    sheet.settings['geometry'] = "spherical"
    sheet.settings['lumen_vol_elasticity'] = 1.e-5

    sheet.settings['barrier_radius'] = 100
    sheet.vert_df['barrier_elasticity'] = 280.0

    sheet.settings['apopto_pattern_kwargs'] = apopto_pattern_kwargs
    sheet.settings['apoptosis'] = apoptosis_settings
    sheet.settings['contraction_lt_kwargs'] = contraction_lt_kwargs
    sheet.settings['delaminate_setting'] = delaminate_settings

    sheet.edge_df['weight'] = 1.
    sheet.edge_df['weight_length'] = sheet.edge_df.weight * \
        sheet.edge_df.length

    sheet.face_df['apoptosis'] = 0
    sheet.face_df['current_traction'] = 0.0
    sheet.face_df['radial_tension'] = 0.0
    sheet.face_df['prefered_perimeter'] = 2 * \
        np.sqrt(sheet.face_df['prefered_area'])
    sheet.face_df['perimeter_elasticity'] = 10.
    sheet.face_df['area_elasticity'] = 1.
    sheet.face_df['apoptosis'] = 0

    sheet.vert_df['viscosity'] = 0.1


def define_fold_position(sheet,
                         fold_number=2,
                         position=[-36, -22, 22, 36]):
    """
    Define fold position in the superegg tissue
    along the z axis

    Parameters
    ----------
    fold_number : number of fold
    position : [min1, max1, min2, max2, ...]
    """
    sheet.face_df['fold'] = 0
    for i in range(fold_number):
        i = i * 2
        sheet.face_df.loc[sheet.face_df[(sheet.face_df.z > position[i])
                                        & (sheet.face_df.z < position[i + 1])].index, 'fold'] = i + 1


def apoptosis_ventral(sheet, code_fold=1):

    fold_cell = sheet.face_df[sheet.face_df.fold == code_fold]

    for i in fold_cell.itertuples():
        proba = 0.6 * np.exp(-(i.phi + 3)**2 / 0.7**2) + \
            0.6 * np.exp(-(i.phi - 3)**2 / 0.7**2)
        aleatory_number = random.uniform(0, 1)
        if aleatory_number < proba:
            sheet.face_df.loc[i.Index, "apoptosis"] = 1


def apoptosis_lateral(sheet, code_fold=1):

    fold_cell = sheet.face_df[sheet.face_df.fold == code_fold]

    for i in fold_cell.itertuples():
        proba = 0.3 * np.exp(-(i.phi + 1.5)**2 / 0.8**2) + \
            0.3 * np.exp(-(i.phi - 1.5)**2 / 0.8**2)
        aleatory_number = random.uniform(0, 1)
        if aleatory_number < proba:
            sheet.face_df.loc[i.Index, "apoptosis"] = 1


def apoptosis_dorsal(sheet, code_fold=1):

    fold_cell = sheet.face_df[sheet.face_df.fold == code_fold]

    for i in fold_cell.itertuples():
        proba = 0.25 * np.exp(-(i.phi)**2 / 0.4 * 2)
        aleatory_number = random.uniform(0, 1)
        if aleatory_number < proba:
            sheet.face_df.loc[i.Index, "apoptosis"] = 1


def define_apoptotic_pattern(sheet, code_fold=1):
    """
    Define apoptotic cell according to a probability to enter in apoptotis
    More apoptotic cell in the ventral side of the tissue than in the dorsal side.
    """
    sheet.face_df.apoptosis = 0
    apoptosis_ventral(sheet, code_fold)
    apoptosis_lateral(sheet, code_fold)
    apoptosis_dorsal(sheet, code_fold)


def decrease_polarity_lateral(sheet, face, parallel_weight, perpendicular_weight):
    edges = sheet.edge_df[sheet.edge_df["face"] == face]
    for index, edge in edges.iterrows():
        angle_ = np.arctan2(
            sheet.edge_df.loc[edge.name, "dx"], sheet.edge_df.loc[
                edge.name, "dz"]
        )
        if (((np.abs(angle_) < np.pi / 6) and (np.abs(angle_) > -np.pi / 6)) or
                ((np.abs(angle_) > -np.pi / 6) and (np.abs(angle_) < np.pi / 6)) or
            ((np.abs(angle_) > 5 * np.pi / 6) and (np.abs(angle_) < 7 * np.pi / 6)) or
                ((np.abs(angle_) < -7 * np.pi / 6) and (np.abs(angle_) > -5 * np.pi / 6))):

            sheet.edge_df.loc[edge.name, "weight"] = perpendicular_weight
        else:
            sheet.edge_df.loc[edge.name, "weight"] = parallel_weight


def decrease_polarity_dv(sheet, face, parallel_weight, perpendicular_weight):
    edges = sheet.edge_df[sheet.edge_df["face"] == face]
    for index, edge in edges.iterrows():
        angle_ = np.arctan2(
            sheet.edge_df.loc[edge.name, "dy"], sheet.edge_df.loc[
                edge.name, "dz"]
        )
        if (((np.abs(angle_) < np.pi / 6) and (np.abs(angle_) > -np.pi / 6)) or
                ((np.abs(angle_) > -np.pi / 6) and (np.abs(angle_) < np.pi / 6)) or
            ((np.abs(angle_) > 5 * np.pi / 6) and (np.abs(angle_) < 7 * np.pi / 6)) or
                ((np.abs(angle_) < -7 * np.pi / 6) and (np.abs(angle_) > -5 * np.pi / 6))):
            sheet.edge_df.loc[edge.name, "weight"] = perpendicular_weight
        else:
            sheet.edge_df.loc[edge.name, "weight"] = parallel_weight


def define_polarity(sheet, parallel_weight, perpendicular_weight):
    sheet.edge_df['id_'] = sheet.edge_df.index

    sheet2 = sheet.extract_bounding_box(y_boundary=(30, 150))
    [decrease_polarity_lateral(
        sheet2, i, parallel_weight, perpendicular_weight) for i in range(sheet2.Nf)]
    for i in (sheet2.edge_df.index):
        sheet.edge_df.loc[sheet.edge_df[sheet.edge_df.id_ == sheet2.edge_df.loc[
            i, 'id_']].index, 'weight'] = sheet2.edge_df.loc[i, 'weight']

    sheet2 = sheet.extract_bounding_box(y_boundary=(-150, -30))
    [decrease_polarity_lateral(
        sheet2, i, parallel_weight, perpendicular_weight) for i in range(sheet2.Nf)]
    for i in (sheet2.edge_df.index):
        sheet.edge_df.loc[sheet.edge_df[sheet.edge_df.id_ == sheet2.edge_df.loc[
            i, 'id_']].index, 'weight'] = sheet2.edge_df.loc[i, 'weight']

    sheet2 = sheet.extract_bounding_box(x_boundary=(-150, -30))
    [decrease_polarity_dv(
        sheet2, i, parallel_weight, perpendicular_weight) for i in range(sheet2.Nf)]
    for i in (sheet2.edge_df.index):
        sheet.edge_df.loc[sheet.edge_df[sheet.edge_df.id_ == sheet2.edge_df.loc[
            i, 'id_']].index, 'weight'] = sheet2.edge_df.loc[i, 'weight']

    sheet2 = sheet.extract_bounding_box(x_boundary=(30, 150))
    [decrease_polarity_dv(
        sheet2, i, parallel_weight, perpendicular_weight) for i in range(sheet2.Nf)]
    for i in (sheet2.edge_df.index):
        sheet.edge_df.loc[sheet.edge_df[sheet.edge_df.id_ == sheet2.edge_df.loc[
            i, 'id_']].index, 'weight'] = sheet2.edge_df.loc[i, 'weight']

    # Pour une cellule apoptotic, toutes les jonctions ont le même poids
    if 'apoptosis' in sheet.face_df.columns:
        for f in sheet.face_df[sheet.face_df.apoptosis == 1].index:
            for e in sheet.edge_df[sheet.edge_df.face == f].index:
                sheet.edge_df.loc[e, 'weight'] = 1.

    if 'is_mesoderm' in sheet.face_df.columns:
        for f in sheet.face_df[sheet.face_df.is_mesoderm == 1].index:
            for e in sheet.edge_df[sheet.edge_df.face == f].index:
                sheet.edge_df.loc[e, 'weight'] = 1.


def open_sheet(dirname, t=0, file_name=None, data_names=['vert', 'edge', 'face', 'cell']):
    """Open hdf5 file

    Open HDF5 file correspond to t time from dirname directory.

    Parameters
    ----------
    directory : str
        complete directory path
    t : int
        time step
    """
    if file_name is None:
        file_name = 'invagination_{:04d}.hf5'.format(t)
    dsets = hdf5.load_datasets(os.path.join(dirname, file_name),
                               data_names=data_names)

    specs = config.geometry.cylindrical_sheet()
    sheet = Sheet('ellipse', dsets, specs)
    return sheet


def depth_calculation(sheet, zmin_, zmax_):

    sheet_fold = sheet.extract_bounding_box(z_boundary=(zmin_, zmax_))

    r = np.mean(np.sqrt(sheet_fold.face_df.x**2 + sheet_fold.face_df.y**2))

    return r


def all_depth_calculation(directory, zmin_=-7, zmax_=7):
    sheet_init = open_sheet(directory, 0)
    depth_init = depth_calculation(sheet_init, zmin_, zmax_)

    depths = []
    for t in range(0, 200):
        try:
            sheet = open_sheet(directory, t)
            depths.append(depth_calculation(sheet, zmin_, zmax_) / depth_init)

        except Exception:
            pass

    return depths


from polarity.polarity import model
from polarity.dynamics import EllipsoidLameGeometry as geom

In [8]:
dsets

NameError: name 'dsets' is not defined

In [9]:
sheet = init_tissue()

FileNotFoundError: file ../examples/super_eggs_polarity.hdf5 not found

In [10]:
sheet.dsets

NameError: name 'sheet' is not defined

In [11]:
specs = {'settings':'lumen_vol':1420.9537579884525,}
sheet.update_specs(specs, reset=True)
geom.update_all(sheet)
solver = QSSolver(with_t1=False, with_t3=False)
res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-4})
Sheet_ellips = sheet_view(sheet, coords=["z", "x"])

SyntaxError: invalid syntax (2159732620.py, line 1)

In [12]:
specs = {'settings' : {'barrier_radius': 10}} 
sheet.update_specs(specs, reset=True)
geom.update_all(sheet)
solver = QSSolver(with_t1=False, with_t3=False)
res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-4})
Sheet_ellips = sheet_view(sheet, coords=["z", "x"])

NameError: name 'sheet' is not defined